In [1]:
import pandas as pd
import numpy as np

In [2]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [3]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"]< 0, 0, selloutData["QTY"])

In [4]:
selloutData[selloutData.QTY >= 0].head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


In [5]:
# 53주차 제거 2

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [7]:
## 연산과 자료형을 활용하여 데이터를 정제하기

In [8]:
def customFunction(inputColumn):
    yearValue = inputColumn / 100
    yearValue = int(yearValue)
    return yearValue

In [9]:
# 데이터를 정제하는 방법 활용 함수화, 연산, 자료형

In [10]:
# 방법 1 (자료형 변환)

In [11]:
selloutData["YEAR"] = (selloutData.YEARWEEK/100).astype(int)

In [12]:
# 방법 2 (함수 활용)

In [13]:
selloutData["YEAR"] = selloutData["YEARWEEK"].apply(customFunction)

In [14]:
# 방법 3 (연산자 활용)

In [15]:
selloutData["YEAR"] = selloutData["YEARWEEK"]//100

In [16]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

In [17]:
refinedData = selloutData[selloutData.WEEK%100 < 53]

In [18]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


In [19]:
sortkey = ['REGIONID', 'PRODUCT', 'YEARWEEK']

In [20]:
sortedData = refinedData.sort_values(sortkey)

In [21]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,1
1757,A00,PRODUCT34,201402,679.0,679.0,2014,2
3125,A00,PRODUCT34,201403,578.0,578.0,2014,3
205,A00,PRODUCT34,201404,532.0,532.0,2014,4
4369,A00,PRODUCT34,201405,516.0,516.0,2014,5


In [22]:
### 기존 인덱스를 제거하고 새로운 인덱스(정렬된)를 적용

In [23]:
step1Data = sortedData.reset_index(drop = True)

In [24]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,1
1,A00,PRODUCT34,201402,679.0,679.0,2014,2
2,A00,PRODUCT34,201403,578.0,578.0,2014,3
3,A00,PRODUCT34,201404,532.0,532.0,2014,4
4,A00,PRODUCT34,201405,516.0,516.0,2014,5


In [25]:
### 롤링함수를 적용가기전에는 정렬을 해줘야한다.

In [26]:
### 1. 기본 이동평균 함수

In [27]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(window = 15, center=True, min_periods=1).mean()

In [28]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000


In [29]:
pwd

'E:\\GitMaster\\01.Python_Git\\04. Data Control'

In [30]:
# groupby 함수를 활용하여 원하는 데이터 정제

In [31]:
filteredData=["REGIONID","PRODUCT","YEAR"]

In [32]:
filteredData = step1Data[(step1Data.REGIONID=="A00")&(step1Data.PRODUCT=="PRODUCT34")]

In [33]:
filteredData.groupby(groupkey)[["NEW_QTY"]].mean()

NameError: name 'groupkey' is not defined

In [34]:
groupkey = ["REGIONID","PRODUCT","YEAR"]

In [35]:
groupData=step1Data.groupby(groupkey)[["NEW_QTY"]].mean()

In [36]:
groupData.columns = ["QTY_MEAN_YEAR"]

In [37]:
groupData.head()

QTY_MEAN_YEAR
REGIONID PRODUCT   YEAR               
A00      PRODUCT34 2014     275.961538
                   2015      86.634615
                   2016      36.576923
         PRODUCT58 2014       2.673077
                   2015       5.711538

In [38]:
groupData = groupData.reset_index()

In [39]:
groupData.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN_YEAR
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


In [40]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000


In [41]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [42]:
mergedData = pd.merge(groupData,step1Data, left_on=joinKey, right_on=joinKey)\
[["REGIONID","PRODUCT","YEARWEEK","YEAR","WEEK","QTY_MEAN_YEAR","NEW_QTY","MA"]]

ValueError: len(right_on) must equal len(left_on)

In [ ]:
mergedData.head()

In [ ]:
len(mergedData)

In [ ]:
leftData = step1Data

In [ ]:
leftData.head(1)

In [ ]:
joinKey2=["REGIONID","PRODUCT"]

In [ ]:
mergedData1 = pd.merge(leftData, groupData, how="left", on=joinKey2)

In [ ]:
mergedData1.head()

In [ ]:
len(mergedData1)

# 그룹바이함수 만들기

In [ ]:
## 제품별로 그룹핑, 제품별 그룹핑을 하지않고 롤링을 하게 되면 원하지 않는 데이터간의 간섭이 일어나 
## 잘못된 데이터를 산출하게 됨
## 이를 막기위해 그룹바이 함수를 활용하여 지역,상품 별 그룹화를 한다

In [ ]:
groupKey = ["REGIONID","PRODUCT"]

In [ ]:
groupDataSet = mergedData.groupby(groupKey)

In [ ]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])

In [ ]:
oneGroup.head()

In [ ]:
def groupRolling(oneGroup):

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5, center=True).mean()
    
    return indexGroupData

In [ ]:
finalResult = mergedData.groupby(groupKey).apply(groupRolling)

In [ ]:
finalResult=finalResult.reset_index(drop=True)

In [ ]:
finalResult.head()

In [ ]:
finalResult.to_csv("./finalResult3.csv")

In [ ]:
fullData = selloutData

In [ ]:
a01Data = fullData[fullData.REGIONID=="A01"]
len(a01Data)

In [ ]:
a02Data = fullData[fullData.REGIONID=="A20"]
len(a02Data)

In [ ]:
resultConcat = pd.concat([a01Data,a02Data], axis=0)

In [ ]:
len(resultConcat)

In [ ]:
## 특정 칼럼 이름 변경

In [ ]:
finalResult = finalResult.rename(columns = {'MEAN_YEAR': 'QTY_MEAN_YEAR','WEEK1':'WEEK'})

In [ ]:
## 특정 칼럼 삭제

In [ ]:
# finalResult 데이터의 상품별 QTY의 마지막 4주차동안의 평균값을 컬럼에 추가

In [ ]:
answer1 = finalResult["YEARWEEK"].max()

In [ ]:
answer1

In [ ]:
reData1 = finalResult[(finalResult.YEARWEEK<=answer1) & (finalResult.YEARWEEK>answer1-4)]

In [ ]:
reData1

In [ ]:
groupkey = ["REGIONID","PRODUCT"]

In [ ]:
eachGroupData = reData1.groupby(groupkey)[["NEW_QTY"]].mean()

In [ ]:
eachGroupData.reset_index().head()

In [ ]:
realFinalData = pd.merge(finalResult, eachGroupData, how="left", on=groupkey)

In [ ]:
realFinalData = realFinalData.rename(columns = {'NEW_QTY_y': 'MEAN'})

In [ ]:
realFinalData